# Diving into Pinecone

In [6]:
# loading the API keys in environment variables
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [7]:
os.environ.get('PINECONE_API_KEY')

'ba0f20ba-b191-4c11-8d77-ebc38f85459c'

In [8]:
import pinecone

# initializing pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

In [9]:
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.4')

### Pinecone Indexes

In [10]:
pinecone.list_indexes()

[]

#### Creating an index

In [15]:
# creating an index
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name} ....')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
    print('Done')
else:
    print(f'Index {index_name} already exists!')

Creating index langchain-pinecone ....
Done


In [12]:
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

#### Deleting an index

In [13]:
# deleting an index
index_name = input('Enter Pinecone index to delete: ')
if index_name in pinecone.list_indexes():
    print(f'Deleting index {index_name} ... ')
    pinecone.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exist!')

Enter Pinecone index to delete:  langchain-pinecone


Deleting index langchain-pinecone ... 
Done


#### Getting index statistics

In [16]:
# getting statistics
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

#### Inserting into an index

In [25]:
# inserting some random vectors into a Pinecone index
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]

# vectors id
ids = list('abcde')

In [26]:
ids

['a', 'b', 'c', 'd', 'e']

In [45]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

#### Updating a vector

In [29]:
# updating a vector
index.upsert(vectors=[('c', [0.3] * 1536)])

{'upserted_count': 1}

#### Fetching a vector

In [30]:
# fetching a vector
index = pinecone.Index('langchain-pinecone')
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'vectors': {'c': {'id': 'c',
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
       

#### Deleting vectors

In [54]:
# deleting vectors
index.delete(ids=['b', 'c'])

{}

In [53]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 3e-05,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [36]:
index.fetch(ids=['b'])

{'namespace': '', 'vectors': {}}

In [52]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.delete(delete_all=True)

{}

#### Querying

In [55]:
# querying 
queries = [[random.random() for _ in range(1536)] for v in range(2)]

In [56]:
index.query(
    queries=queries,
    top_k=3,
    include_values=False
)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '80', 'x-pinecone-request-latency-ms': '90', 'date': 'Mon, 15 Jan 2024 12:47:31 GMT', 'x-envoy-upstream-service-time': '2', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":3,"message":"The 'queries' parameter has been deprecated.","details":[]}
